In [24]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

In [25]:
data1 = pd.read_excel('2c治疗方案.xlsx') # 治疗方案
data2 = pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx') # 水肿数据
data3 = pd.

SyntaxError: invalid syntax (2637392360.py, line 3)

In [26]:
data3 = pd.merge(data1, data2, how='outer', on='ID')

data3 = data3.drop(range(101, 131))

In [27]:
data3.columns

Index(['ID', '脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经',
       '发病到首次影像检查时间间隔', '入院首次影像检查流水号', 'ED_volume.0', '随访1流水号', 'ED_volume.1',
       '随访2流水号', 'ED_volume.2', '随访3流水号', 'ED_volume.3', '随访4流水号',
       'ED_volume.4', '随访5流水号', 'ED_volume.5', '随访6流水号', 'ED_volume.6',
       '随访7流水号', 'ED_volume.7', '随访8流水号', 'ED_volume.8'],
      dtype='object')

In [28]:
import json
with open('best_labels.json','r') as f:
    groups_label=json.load(f)
print(len(groups_label['best_labels']))

KeyError: 'best_labels'

In [29]:
dataT=data3.head(100)
dataT['分组信息']=groups_label['best_labels']
dataT.to_excel('含分组信息数据.xlsx',index=False)

KeyError: 'best_labels'

In [30]:
# 创建一个新列"差"，初始化为0
dataT['差'] = 0
U=globals()
res={}
for i in list(set(dataT['分组信息'])):
    U[f'data_g{i}']=dataT[dataT['分组信息']==i]

# 遍历数据集的每一行
    for index, row in U[f'data_g{i}'].iterrows():
        row_sum = 0  # 用于累积每行的差值

        prev_value = None  # 用于存储前一个以"ED"开头的列的值

        # 遍历以"ED"开头的列
        for col in row.index:
            if col.startswith('ED'):
                value = row[col]
                if not pd.isna(value):  # 确保元素不是NaN
                    if prev_value is not None:
                        row_sum += (value - prev_value)  # 计算差值并累积
                    prev_value = value

        # 根据row_sum的值设置"差"列的值
        if row_sum > 0:
            U[f'data_g{i}'].at[index, '差'] = 1
        elif row_sum < 0:
            U[f'data_g{i}'].at[index, '差'] = -1
        res[f'组{i}']=float(U[f'data_g{i}']['差'].sum())

res_res = {}
for i in res:
    if res[i] < 0:
        res_res[i]=-1
    else:
        res_res[i]=1
# with open('分组趋势.json', 'w') as f:
#     json.dump(res_res,f)

KeyError: '分组信息'

In [ ]:
# with open('分组趋势.json', 'r') as f:
#     print(json.load(f))

In [31]:

for i in res_res:
    

SyntaxError: incomplete input (1160186974.py, line 2)

In [32]:
list(data1.columns)

['ID', '脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']